# VideoStream 视频流

VideoStream组件可以显示来自本地流（例如网络摄像头）的视频，并允许从Python访问流式视频数据。

底层实现为`panel.widgets.VideoStream`，参数基本一致，参考文档：https://panel.holoviz.org/reference/widgets/VideoStream.html


In [1]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

视频流组件默认情况下会显示视频流，可用于如网络摄像头实时视频的展示。


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnVideoStream name="视频流" />
</template>


## 截图功能

可以调用`snapshot`方法触发组件的`value`更新，以获取当前视频帧的图像。


In [12]:
##ignore
import panel as pn
pn.extension()

video_stream = pn.widgets.VideoStream(name='Video Stream')
video_stream.snapshot()

html = pn.pane.HTML(width=320, height=240)

def update(event):
    html.object = '<img src="'+event.new+'" width=320 height=240 />'
    
video_stream.param.watch(update, 'value')

html

HTML(None, height=240, sizing_mode='fixed', width=320)

In [16]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnCol>
    <PnVideoStream 
      name="video stream" 
      @change="update_snapshot"
      v-model="snapshot_img.value"
      ref="video_stream_ref"
    />
    <PnButton 
      name="Snapshot" 
      button_type="primary" 
      @click="take_snapshot()"
    />
  </PnCol>
  <img alt='snap' :src="snapshot_img.value" />
  <PnHTML :object="snapshot_html.value" width='320' height='240' />
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

snapshot_html = ref("")
video_stream_ref = ref(None)
snapshot_img = ref('')

def take_snapshot():
    if video_stream_ref.value:
        print('img', video_stream_ref.value.unwrap().value)
        video_stream_ref.value.unwrap().snapshot()

def update_snapshot(event):
    print(event.new)
    if event.new:
        snapshot_html.value = f'<img src="{event.new}" width=320 height=240 />'
        # video_stream_ref.value = event['owner']
</script>

{"vue": "<template>\n  <PnCol>\n    <PnVideoStream \n      name=\"video stream\" \n      @change=\"update_snapshot\"\n      v-model=\"snapshot_img.value\"\n      ref=\"video_stream_ref\"\n    />\n    <PnButton \n      name=\"Snapshot\" \n      button_type=\"primary\" \n      @click=\"take_snapshot()\"\n    />\n  </PnCol>\n  <img alt='snap' :src=\"snapshot_img.value\" />\n  <PnHTML :object=\"snapshot_html.value\" width='320' height='240' />\n</template>\n<script lang='py'>\nimport panel as pn\nfrom vuepy import ref\n\nsnapshot_html = ref(\"\")\nvideo_stream_ref = ref(None)\nsnapshot_img = ref('')\n\ndef take_snapshot():\n    if video_stream_ref.value:\n        print('img', video_stream_ref.value.unwrap().value)\n        video_stream_ref.value.unwrap().snapshot()\n\ndef update_snapshot(event):\n    print(event.new)\n    if event.new:\n        snapshot_html.value = f'<img src=\"{event.new}\" width=320 height=240 />'\n        # video_stream_ref.value = event['owner']\n</script>\n", "setup"


## 定时截图

通过设置`timeout`参数，可以指定视频流将以多大频率更新。


In [3]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnCol>
    <PnToggle name="暂停" v-model="paused.value" />
    <PnRow>
      <PnVideoStream 
        :timeout="1000" 
        :paused="paused.value"
        @change="update_timed_snapshot"
      />
    </PnRow>
  </PnCol>
  <PnHTML :object="timed_html.value" />
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

pn.extension()

paused = ref(False)
timed_html = ref("")

def update_timed_snapshot(event):
    if 'value' in event and event.new:
        timed_html.value = f'<img src="{event.new}" width=320 height=240 />'
</script>

{"vue": "<template>\n  <PnCol>\n    <PnToggle name=\"\u6682\u505c\" v-model=\"paused.value\" />\n    <PnRow>\n      <PnVideoStream \n        :timeout=\"1000\" \n        :paused=\"paused.value\"\n        @change=\"update_timed_snapshot\"\n      />\n    </PnRow>\n  </PnCol>\n  <PnHTML :object=\"timed_html.value\" />\n</template>\n<script lang='py'>\nimport panel as pn\nfrom vuepy import ref\n\npn.extension()\n\npaused = ref(False)\ntimed_html = ref(\"\")\n\ndef update_timed_snapshot(event):\n    if 'value' in event and event.new:\n        timed_html.value = f'<img src=\"{event.new}\" width=320 height=240 />'\n</script>\n", "setup": ""}



## 图像格式

可以通过`format`参数指定捕获的图像格式，如果需要高频率的截图，可以选择'jpeg'格式，因为图像尺寸要小得多。


In [5]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnCol>
    <PnRow>
      <PnVideoStream 
        name="PNG格式" 
        format="png"
        :timeout="500"
        :width="320"
        :height="240"
      />
      <PnVideoStream 
        name="JPEG格式" 
        format="jpeg"
        :timeout="500"
        :width="320"
        :height="240"
      />
    </PnRow>
    <PnStaticText value="JPEG格式适合高频率截图，因为图像尺寸更小" />
  </PnCol>
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

pn.extension()
</script>

{"vue": "<template>\n  <PnCol>\n    <PnRow>\n      <PnVideoStream \n        name=\"PNG\u683c\u5f0f\" \n        format=\"png\"\n        :timeout=\"500\"\n        :width=\"320\"\n        :height=\"240\"\n      />\n      <PnVideoStream \n        name=\"JPEG\u683c\u5f0f\" \n        format=\"jpeg\"\n        :timeout=\"500\"\n        :width=\"320\"\n        :height=\"240\"\n      />\n    </PnRow>\n    <PnStaticText value=\"JPEG\u683c\u5f0f\u9002\u5408\u9ad8\u9891\u7387\u622a\u56fe\uff0c\u56e0\u4e3a\u56fe\u50cf\u5c3a\u5bf8\u66f4\u5c0f\" />\n  </PnCol>\n</template>\n<script lang='py'>\nimport panel as pn\nfrom vuepy import ref\n\npn.extension()\n</script>\n", "setup": ""}



## API

### 属性

| 属性名 | 说明 | 类型 | 默认值 |
| -------- | ------------------- | ---------------------------------------------------------------| ------- |
| format | 捕获图像的格式，'png'或'jpeg' | ^[string] | 'png' |
| paused | 视频流是否暂停 | ^[boolean] | false |
| timeout | 截图之间的间隔（毫秒），如果为None则仅在调用snapshot方法时才拍摄截图 | ^[int]^[None] | None |
| value | 当前截图的字符串表示 | ^[string] | — |
| snapshot | 触发截图的动作 | ^[boolean] | false |
| name | 组件标题 | ^[string] | — |

### Events

| 事件名 | 说明 | 类型 |
| --- | --- | --- |
| change | 当组件状态（特别是value）改变时触发 | ^[Callable]`(event: dict) -> None` |

### Slots

| 插槽名 | 说明 |
| --- | --- |
| | |

### 方法

| 属性名 | 说明 | 类型 |
| --- | --- | --- |
| | | |


In [6]:
##ignore
import panel as pn
pn.extension()

video_stream = pn.widgets.VideoStream(name='Video Stream')

pn.Row(video_stream.controls(jslink=True), video_stream)

Row
    [0] Tabs
        [0] WidgetBox(margin=(5, 10), name='Controls')
            [0] StaticText(value='<b>Controls</b>')
            [1] TextInput(description='A base64 representation o..., name='Value')
            [2] Checkbox(name='Disabled')
            [3] Select(description='The file format a..., name='Format', options=OrderedDict([('png', ...]), value='png')
            [4] Checkbox(name='Paused')
            [5] FloatInput(description='Interval between s..., name='Timeout')
        [1] WidgetBox(margin=(5, 10), name='Layout')
            [0] StaticText(value='<b>Layout</b>')
            [1] LiteralInput(description='Whether the object should..., name='Align', serializer='json', value='start')
            [2] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', serializer='json')
            [3] ListInput(description='CSS classes t..., name='Css classes', serializer='json', type=<class 'list'>)
            [4] IntInput(description='The height of the compone..., name='Height', start=0)
            [5] IntInput(description='Minimal width o..., name='Min width', start=0)
            [6] IntInput(description='Minimal height o..., name='Min height', start=0)
            [7] IntInput(description='Maximum width o..., name='Max width', start=0)
            [8] IntInput(description='Maximum height o..., name='Max height', start=0)
            [9] LiteralInput(description='Allows to create addition..., name='Margin', serializer='json', value=(5, 10))
            [10] DictInput(description='Dictionary of C..., name='Styles', serializer='json', type=<class 'dict'>)
            [11] ListInput(description='List of arbitrary t..., name='Tags', serializer='json', type=<class 'list'>)
            [12] IntInput(description='The width of the componen..., name='Width', start=0)
            [13] Select(description='Describes how the compone..., name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
            [14] Select(description='Describes how the compone..., name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
            [15] Select(description='How the component should ..., name='Sizing mode', options=OrderedDict([('fixed', ...]))
            [16] Checkbox(name='Visible', value=True)
    [1] VideoStream(name='Video Stream')